In [1]:
import yaml
import re
import os

#Set to your local directory 
yaml_file = 'C:/Users/jocke/GitHub/PattonSteel/Misc/Brewery/DrinkConfig.yaml'
destination = 'C:/Users/jocke/GitHub/PattonSteel/Misc/Brewery/FlatFiles/'

In [2]:
class Recipe:
    def __init__(self, name: str, cookingtime: int, ingredients: list, difficulty: int, **kwargs):
        # required params
        self.name = name
        self.cookingtime = cookingtime
        self.ingredients = ingredients
        self.difficulty = difficulty

        # optional params
        self.distilltime = kwargs.get('distilltime', None)
        self.distillruns = kwargs.get('distillruns', None)
        self.wood = kwargs.get('wood', None)
        self.age = kwargs.get('age', None)
        self.alcohol = kwargs.get('alcohol', None)
        self.effects = kwargs.get('effects', None)

    def difficulty_word(self):
        difficulty_levels = ["rather simple","relatively straightforward","easy","of moderate difficulty"
            ,"of intermediate difficulty","somewhat challenging","rather difficult","very complex","quite hard","one of the most difficult"]
        return difficulty_levels[self.difficulty-1]
    
    def wood_word(self):
        wood_types = [" birch","n oak"," jungle"," spruce","n acacia"," dark oak"," crimson"," warped"," mangrove"," cherry"," bamboo"," cut copper"]
        return wood_types[self.wood-1]

In [3]:
#Read in YML config file, store in data, if there is an error calling this function, check your yaml file for un-indented items
def parse_yaml():
    recipes:list[Recipe] = []

    with open(yaml_file, 'r') as file:
        data = yaml.safe_load(file)

    for recipe in data:
        current_recipe = Recipe(
            name=recipe,
            cookingtime=int(data[recipe]["cookingtime"]),
            ingredients=data[recipe]["ingredients"],
            difficulty=int(data[recipe]["difficulty"]),
        )

        for key in ['distilltime', 'distillruns', 'wood', 'age', 'alcohol', 'effects']:
            if key in data[recipe]:
                setattr(current_recipe, key, data[recipe][key])

        recipes.append(current_recipe)
    
    return recipes

In [4]:
#Generates book given a particlar recipe
def generate_book(recipe:Recipe):
    seperated_name=re.sub(r'([a-z])([A-Z])', r'\1 \2', recipe.name)

    # Page 1 - Potion, Effects
    content="<AutoPage><AutoAlias><ReqPermission>\n"
    content+=f"{seperated_name} &6[R]&0\n\n"
    if recipe.effects!=None:
        content+="Effects:\n"
        for effects in recipe.effects:
            content+=f"  &o{effects.split('/')[0].lower()}\n"
    content+=f"\nIt is {recipe.difficulty_word()} to create!\n\n"
    
    # Page 2 - Ingredients, Ferment Time
    content+=f"<NextPage>\n"
    content+=f"{seperated_name} &6[R]&0\n\n"
    content+=f"Ingredients:\n"
    for ingredient in recipe.ingredients:
        to_print = f"{ingredient.split('/')[1]} {ingredient.split('/')[0]}"
        to_print = to_print.replace("_"," ")
        to_print = to_print.replace("Brewery:","&6&o")
        content+=f"  &o{to_print}\n"
    content+=f"&0&o\nFerment for {recipe.cookingtime} min(s)\n"
        
    # Page 3 - Distilling, Aging
    content+=f"<NextPage>\n"
    content+=f"{seperated_name} &6[R]&0\n\n"
    ## distilling
    if recipe.distillruns!=None:
        content+=f"Distill this brew {recipe.distillruns} time(s) for {recipe.distilltime} seconds or so...\n\n"
    else:
        content+=f"No distilling is required for this brew.\n\n"

    ## aging
    if recipe.wood !=None:
        if recipe.wood == 0:
            content+=f"One can use any type of barrel, but must age for {recipe.age} days.\n\n"
        else:
            content+=f"It is best to age with a{recipe.wood_word()} wood barrel for {recipe.age} days.\n\n"
    else:
        content+=f"No aging is required for this brew.\n\n"

            
    if recipe.alcohol!=None:
        content+=f"Properly brewed, this will have an alcohol density of {recipe.alcohol}."

    file_name = seperated_name.replace(' ','_')
    file_path = f"{destination}{file_name}_&6[R].txt"
    with open(file_path,'w',newline="\n") as file:
        file.write(content)

In [7]:
#Call function to convert yaml into Recipe objects
recipes:list[Recipe] = parse_yaml()
#Iterate through Recipes, creating a book for each
for recipe in recipes:
    try:
        generate_book(recipe)
    except Exception as e:
        print(f"There was an issue creating {recipe.name}! Please take a look.")
        print(e)

ParserError: while parsing a block mapping
  in "C:/Users/jocke/GitHub/PattonSteel/Misc/Brewery/DrinkConfig.yaml", line 2, column 5
expected <block end>, but found '<anchor>'
  in "C:/Users/jocke/GitHub/PattonSteel/Misc/Brewery/DrinkConfig.yaml", line 2, column 13